In [6]:
import requests
from datetime import datetime

LONG, LAT = (39.5789544215873, -74.22461197107408)


def get_wind_data(latitude=LAT, longitude=LONG, hours=24, days=3):
    # url = (
    #     "https://marine-api.open-meteo.com/v1/marine?"
    #     "latitude=39.565&longitude=-74.240&hourly=wind_speed_10m,wind_direction_10m&"
    #     "daily=wind_speed_10m_max,wind_direction_10m_dominant&timezone=auto"
    # )
    base_url = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "wind_speed_10m,wind_direction_10m",
        "daily": "wind_speed_10m_max,wind_direction_10m_dominant",
        "timezone": "auto"
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()

    # Get current hour index
    current_hour = datetime.now().strftime("%Y-%m-%dT%H:00")
    hour_idx = data["hourly"]["time"].index(current_hour)

    current_wind_speed = data["hourly"]["wind_speed_10m"][hour_idx]
    current_wind_dir = data["hourly"]["wind_direction_10m"][hour_idx]

    # Next 24 hours
    next_24 = {
        "wind_speeds": data["hourly"]["wind_speed_10m"][hour_idx : hour_idx + 24],
        "wind_dirs": data["hourly"]["wind_direction_10m"][hour_idx : hour_idx + 24],
        "times": data["hourly"]["time"][hour_idx : hour_idx + 24],
    }

    # Next 3 days
    next_3_days = list(
        zip(
            data["daily"]["time"][:3],
            data["daily"]["wind_speed_10m_max"][:3],
            data["daily"]["wind_direction_10m_dominant"][:3],
        )
    )

    return {
        "current": {"speed": current_wind_speed, "dir": current_wind_dir},
        "next_24": next_24,
        "next_3_days": next_3_days,
    }


get_wind_data()

ValueError: '2025-07-03T20:00' is not in list